In [6]:
!git clone https://github.com/DanielLiangAjj/DBMI_research_profile_crawler.git

fatal: destination path 'DBMI_research_profile_crawler' already exists and is not an empty directory.


In [7]:
!pip install nltk rouge-score

In [28]:
overview = []
for filename in os.listdir('/content/DBMI_research_profile_crawler/Research Overview'):
    if filename.endswith('.txt'):
        name, _ = os.path.splitext(filename)  # Correct usage of os.path.splitext
        overview.append(name)

files = []
for filename in os.listdir('/content/DBMI_research_profile_crawler/researchers_files(Yilu_format)'):
    if filename.endswith('.json'):
        name, _ = os.path.splitext(filename)  # Corrected os.path.splitext
        files.append(name)

# Checking differences between overview and files
for i in overview:
    if i not in files:
        print(i, end=", ")
print()
print('============================')
for i in files:
    if i not in overview:
        print(i, end=", ")


David Bickers, Anthony W P Fitzpatrick, Kimberly Kwei, Susan F Steinberg, Noemie Elhadad, Rogerio Lobo, Andrew Chan, Elizabeth Oelsner, Mitchell Elkind, Dritan Agalliu, Howard Lieberman, Gulgun Tezel, Anil Lalwani, Hasan Abaci, Andrew Marks, Lawrence Lenke, Patrick Ryan, Jason Wright, Jonathan Barasch, Charles Emala, Chris Wiggins, Joel Lavine, Peter Kwong, Dani Dumitriu, Elaine Wan, Mary D'Alton, Lori Zeltser, Kapil Ramachandran, Liza Pon, Paul Sajda, Emmanuelle Passegue, Munemasa Mori, Melissa Stockwell, B Timothy Walsh, Richard Baer, Christine Kim, Joel Stein, Marwah Abdalla, Andres Bendesky, Aurel Lazar, Adam Mor, David Fidock, George Hripcsak, Muredach Reilly, Suzanne Lentzsch, Markus Siegelin, Max E Gottesman, Serge Cremers, Churl-Su Kwon, Shirley ShiDu Yan, Anne-Catrin Uhlemann, Xiao Xu, Julie Canman, Howard Worman, S Abid Hussaini, Isabelle Schrauwen, Brian Gill, Radhika Jagannathan, Peter St George-Hyslop, David Kalfa, Brent Stockwell, Franklin Costantini, Elana Bernstein, Rui

# **Data Preparation & Parsing**

In [31]:
import nltk
import json
import re
import csv
import os
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer

nltk.download('punkt')
candidate_path = '/content/DBMI_research_profile_crawler/Research Overview'
comparison_csv_path = '/content/DBMI_research_profile_crawler/columbia_research_faculty_extracted.csv'

# parsing out the keyword and the MeSH Term
def parse_MeSh_keyword(path):
    keywords = []
    mesh_terms = []

    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)

    for article in data:
        keywords.extend(article.get('Keywords', []))
        mesh_terms.extend(article.get('MeSH terms', []))

    return keywords, mesh_terms

# def normalize_name(name):
#     # Remove titles like "PhD", "MD", etc.
#     name = re.sub(r',?\s*(PhD|MD|Dr|Prof|MS|mfa|ph.d|d.|mph|msw|mba|cgc|l|mbe|ma|otr/l|bcb|m.|llb|ch.b|gpd)\.?', '', name, flags=re.IGNORECASE)
#     # Remove middle names/initials
#     name = re.sub(r'\b[A-Z]\.\b', '', name)
#     # Remove extra whitespace and convert to lower case
#     name = re.sub(r'\s+', ' ', name).strip().lower()
#     return name

# function to get rid of the titles following after the name
def normalize_name(name):
    if "," in name:
        comma_index = name.find(",")
        return name[:comma_index]
    return name

comparison_names = {}
# read from the csv file and extract the researcher name with their research introduction scraped by the web scraper
with open(comparison_csv_path, mode='r', newline='', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        if row['Research Introduction'] != 'N/A':
            comparison_names[row['Name']] = row['Research Introduction']
# scraped content
normalized_comparison_names = {normalize_name(name): intro for name, intro in comparison_names.items()}
# print(normalized_comparison_names.keys())
file_names = []
# GPT summarized content based on Keywords and MeSH Terms
file_names_dict = {}
for filename in os.listdir(candidate_path):
    if filename.endswith('.txt'):
        name, _ = os.path.splitext(filename)
        # file_names.append(name.lower())
        file_names.append(name)
        # name = name.lower()
        name = name.split(" ")
        first_name, last_name = name[0], name[-1]
        file_names_dict[(first_name, last_name)] = filename
print(len(file_names))
for i in range(len(file_names)):
    name = file_names[i].split(" ")
    first_name, last_name = name[0], name[-1]
    file_names[i] = (first_name, last_name)

matches = {}
for i in normalized_comparison_names:
    name = i.split(" ")
    if (name[0], name[-1]) in file_names and normalized_comparison_names[i] != 'N/A':
        matches[i] = normalized_comparison_names[i]
len(matches)


671


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


172

# **BLEU & ROUGE**

In [32]:
for test_name in matches:
    test_name_copy = test_name
    test_name = test_name.split(" ")
    test_first_name, test_last_name = test_name[0], test_name[-1]
    test_path = candidate_path+'/'+file_names_dict[(test_first_name, test_last_name)]
    with open(test_path, 'r', encoding='utf-8') as file:
        test_contents = file.read()
    # print(matches[test_name_copy])
    # Tokenize the paragraphs into sentences
    reference_sentences = nltk.sent_tokenize(matches[test_name_copy])
    candidate_sentences = nltk.sent_tokenize(test_contents)

    # Tokenize sentences into words
    reference_tokens = [nltk.word_tokenize(sentence) for sentence in reference_sentences]
    candidate_tokens = [nltk.word_tokenize(sentence) for sentence in candidate_sentences]

    # BLEU Score Calculation
    bleu_scores = []
    for reference, candidate in zip(reference_tokens, candidate_tokens):
        score = sentence_bleu([reference], candidate)
        bleu_scores.append(score)

    average_bleu_score = sum(bleu_scores) / len(bleu_scores)
    print(f"Average BLEU score for {test_name_copy}: {average_bleu_score}")

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = []
    for reference, candidate in zip(reference_sentences, candidate_sentences):
        scores = scorer.score(reference, candidate)
        rouge_scores.append(scores)

    # Calculate average ROUGE scores
    average_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    average_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    average_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)
    print(f"ROUGE Score for {test_name_copy}:")
    print(f"Average ROUGE-1 score: {average_rouge1:.2f}")
    print(f"Average ROUGE-2 score: {average_rouge2:.2f}")
    print(f"Average ROUGE-L score: {average_rougeL:.2f}")
    print("==================================================")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU score for Hashim M. Al-Hashimi: 0.0235269071488911
ROUGE Score for Hashim M. Al-Hashimi:
Average ROUGE-1 score: 0.28
Average ROUGE-2 score: 0.06
Average ROUGE-L score: 0.16
Average BLEU score for Richard Axel: 9.684103801621567e-232
ROUGE Score for Richard Axel:
Average ROUGE-1 score: 0.08
Average ROUGE-2 score: 0.00
Average ROUGE-L score: 0.07
Average BLEU score for Bridget Carragher: 1.1001926558807207e-155
ROUGE Score for Bridget Carragher:
Average ROUGE-1 score: 0.15
Average ROUGE-2 score: 0.01
Average ROUGE-L score: 0.11
Average BLEU score for Anum Glasgow: 5.719256820821495e-156
ROUGE Score for Anum Glasgow:
Average ROUGE-1 score: 0.18
Average ROUGE-2 score: 0.00
Average ROUGE-L score: 0.12
Average BLEU score for Stephen P. Goff: 1.0510731323279964e-231
ROUGE Score for Stephen P. Goff:
Average ROUGE-1 score: 0.13
Average ROUGE-2 score: 0.01
Average ROUGE-L score: 0.11
Average BLEU score for Max E. Gottesman: 1.0446815961025794e-231
ROUGE Score for Max E. Gottesman:
A

# **Rhetorical Sentence Parsing (Word Mover's Distance, Cosine Similarity, Sentiment and Tone Analysis)**

In [40]:
!pip install vaderSentiment
!pip install POT

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 835.4/835.4 kB 52.4 MB/s eta 0:00:00


In [42]:
import nltk
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import numpy as np
import gensim.downloader as api

# Load pre-trained GloVe embeddings
word_vectors = api.load("glove-wiki-gigaword-300")

# Initialize sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Function to calculate Word Mover's Distance for paragraphs
def calculate_wmd(paragraph1, paragraph2):
    paragraph1_tokens = [word for word in nltk.word_tokenize(paragraph1.lower()) if word in word_vectors]
    paragraph2_tokens = [word for word in nltk.word_tokenize(paragraph2.lower()) if word in word_vectors]
    return word_vectors.wmdistance(paragraph1_tokens, paragraph2_tokens)

# Function to calculate average word vectors for cosine similarity for paragraphs
def average_word_vectors(paragraph, model, num_features):
    words = nltk.word_tokenize(paragraph.lower())
    feature_vec = np.zeros((num_features,), dtype="float32")
    n_words = 0
    for word in words:
        if word in model:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if n_words > 0:
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

# Function to analyze sentiment for paragraphs
def analyze_sentiment(paragraph):
    return analyzer.polarity_scores(paragraph)

In [46]:
# Iterate through the matches
counter=0
for test_name in matches:
    test_name_copy = test_name
    test_name = test_name.split(" ")
    test_first_name, test_last_name = test_name[0], test_name[-1]
    test_path = candidate_path+'/'+file_names_dict[(test_first_name, test_last_name)]
    with open(test_path, 'r', encoding='utf-8') as file:
        test_contents = file.read()

    # Process the entire paragraph
    reference_paragraph = matches[test_name_copy]
    candidate_paragraph = test_contents

    print(f"Processing paragraph for {test_name_copy}:")
    print(f"Reference Paragraph (Human Written Scraped Content): {reference_paragraph}")
    print(f"Candidate Paragraph (GPT Generated Content): {candidate_paragraph}")
    # Detecting Subtle Differences in Meaning (Word Mover's Distance)
    wmd_distance = calculate_wmd(reference_paragraph, candidate_paragraph)
    print(f"-->Word Mover's Distance between paragraphs: {wmd_distance}")

    # Cosine Similarity using Word Embeddings for Paragraphs
    reference_vector = average_word_vectors(reference_paragraph, word_vectors, 300)
    candidate_vector = average_word_vectors(candidate_paragraph, word_vectors, 300)
    cosine_sim = cosine_similarity([reference_vector], [candidate_vector])[0][0]
    print(f"-->Cosine Similarity for paragraphs: {cosine_sim}")

    # Sentiment and Tone Analysis for Paragraphs
    reference_sentiment = analyze_sentiment(reference_paragraph)
    candidate_sentiment = analyze_sentiment(candidate_paragraph)
    print("Sentiment Analysis for paragraphs:")
    print(f"-->Reference Paragraph Sentiment: {reference_sentiment}")
    print(f"-->Candidate Paragraph Sentiment: {candidate_sentiment}")

    # Compare sentiment differences
    sentiment_diff = compare_sentiment(reference_sentiment, candidate_sentiment)
    print(f"-->Sentiment Differences for paragraphs: {sentiment_diff}")
    print("==================================================")
    if counter == 0:
        break


Processing paragraph for Hashim M. Al-Hashimi:
Reference Paragraph (Human Written Scraped Content): Al-Hashimi is interested in developing a deep, quantitative, and predictive understanding of cellular processes based on the fundamental behaviors of nucleic acids and their interactions with protein binding partners.  Over the past two decades, Al-Hashimi and his trainees developed approaches combining NMR spectroscopy, computational modeling, optical melting experiments, and chemical probing to determine 3D dynamic ensembles of RNA and DNA molecules at atomic resolution.  Using dynamic ensembles of nucleic acids, the Al-Hashimi group has developed quantitative and predictive models for several fundamental biological processes, including DNA replication fidelity, Tat-dependent HIV-1 transcriptional activation, RNA folding, and the impact of post-transcriptional modifications such as m6A and Nm on translation, splicing, and RNA-protein interactions.  These studies have reshaped structura